# Task 2 - Classic CV - Drone navigation

For drone navigation and mapping tasks, choosing the right algorithms for image processing is essential. I started with a simple baseline approach by converting all images to grayscale and matching them directly, **without interpolation or smoothing**.

Here, I decided to use feature extraction, matching, and geometric transformations (**Homography**) due to their strong suitability for aerial imagery analysis.

### **Feature Extraction and Matching (SIFT + FLANN)**

I chose **SIFT** (Scale-Invariant Feature Transform) because drone images often have varying scales, rotations, and lighting conditions. SIFT creates special "keypoints" (distinctive image features) that stay consistent even under these changes.

To quickly and accurately match these keypoints between drone images and the global map, I used **FLANN** (Fast Library for Approximate Nearest Neighbors). FLANN is efficient at matching large sets of descriptors.

**Pros of SIFT + FLANN:**
- robust to rotation, scaling, and illumination changes;
- highly accurate matching;
- well-supported and widely used;
- FLANN is fast enough even for Google Colab.

**Cons of SIFT + FLANN:**
- computationally heavier compared to simpler binary descriptors like ORB or AKAZE;
- less ideal if real-time speed is critical.

Despite the higher computation, SIFT offered the accuracy needed to reliably reconstruct the drone's flight path.

### **Geometric Matching (Homography + RANSAC)**

After feature matching, I applied **Homography** (a transformation mapping points from one image onto another) to precisely place drone images on the satellite map. For calculating homography, I used **RANSAC** (Random Sample Consensus), which filters out incorrect matches ("outliers") and gives reliable results.

**Pros of Homography + RANSAC:**
- robust against incorrect matches and noise (Fischler & Bolles, 1981);
- provides precise position and orientation information;
- highly effective for aerial image alignment.

**Cons of Homography + RANSAC:**
- accuracy strongly depends on the quality of initial matches;
- increased computational load if many incorrect matches are present.

Overall, combining **SIFT, FLANN, and RANSAC Homography** provided the best balance of robustness, accuracy, and efficiency, suitable for performing this UAV navigation task within the constraints of Google Colab.

## 0. Libraries

In [1]:
import cv2
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import re

## 1. Define paths and load data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

global_map_path = '/content/drive/My Drive/Colab Notebooks/drone_navigation_project/data/global_map.png'
crops_folder = '/content/drive/My Drive/Colab Notebooks/drone_navigation_project/data/crops/'
output_folder = '/content/drive/My Drive/Colab Notebooks/drone_navigation_project/output/'

os.makedirs(output_folder, exist_ok=True)

# Load the global map and create a grayscale version.
global_map = cv2.imread(global_map_path)
global_map_gray = cv2.cvtColor(global_map, cv2.COLOR_BGR2GRAY)

Mounted at /content/drive


### Extract crop number from file name.

In [3]:
def get_crop_number(file_path):
    filename = os.path.basename(file_path)
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else -1

# Load and correctly sort all crop images
crop_files = sorted(glob.glob(os.path.join(crops_folder, '*.png')), key=get_crop_number)
drone_crops = [cv2.imread(file) for file in crop_files]

## 2. Setup SIFT and FLANN matcher for feature detection.


I chose SIFT because its scale- and rotation-invariant features work well under varying conditions, which is essential for UAV photos. I compute the keypoints and descriptors on the global map once and then use FLANN for matching since it is both efficient and effective for high-dimensional descriptor spaces. I also set a minimum match count and defined a margin for later visualization

In [4]:
# Initialize SIFT detector
sift = cv2.SIFT_create()
# Compute keypoints and descriptors for the global map
kp_global, des_global = sift.detectAndCompute(global_map_gray, None)

# Matching parameters:
MIN_MATCH_COUNT = 10
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
# Create the FLANN based matcher
flann = cv2.FlannBasedMatcher(index_params, search_params)

# Margin (in pixels) to enlarge the region for visualization
margin = 50

## 3. Prepare for trajectory video output

To visualize the UAV's flight path, I need to store the drone positions computed for each crop image. I also set up the video writer to create an output video. I use the dimensions of the global map for the video frames and choose a suitable frame rate

In [5]:
drone_positions = []  # List to store computed drone positions.

frame_height, frame_width = global_map.shape[:2]
fps = 1
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_output_file = os.path.join(output_folder, 'drone_trajectory.avi')
video_writer = cv2.VideoWriter(video_output_file, fourcc, fps, (frame_width, frame_height))

# Define a fixed frame size for the zoomed-in video
zoom_frame_size = (600, 600)
# Create the video writer for the zoomed region video
zoom_video_output_file = os.path.join(output_folder, 'drone_trajectory_zoomed.avi')
zoom_video_writer = cv2.VideoWriter(zoom_video_output_file, fourcc, fps, zoom_frame_size)

## 4. Process each crop image: matching, homography, rotation, similarity, visualization, and video frame creation.

This step is the core of the pipeline. Each crop image taken by the drone is processed to determine its position and orientation on the global satellite map.

Result:
- drone_trajectory - the full trajectory on the global map, with a triangle indicating the drone's heading;
- drone_trajectory_zoomed - a zoomed-in view that shows only the local map region where the drone is in that frame.



### Draw a rotated triangle representing the drone.

In [6]:
def draw_drone_triangle(frame, center, angle_deg, size=20, color=(0,255,255)):
    """
    Draws a filled triangle on frame with its tip at the given center,
    rotated by angle_deg. Angle 0 means the triangle (by default)
    points upward.

    Parameters:
      - frame: the image on which to draw.
      - center: (x, y) tuple of the drone's location.
      - angle_deg: rotation angle in degrees.
      - size: controls the triangle's size.
      - color: color for the triangle.
    """
    # Define an isosceles triangle in local coordinates
    pts = np.array([[0, -size],
                    [-size // 2, size // 2],
                    [ size // 2, size // 2]], dtype=np.float32)

    theta = np.radians(angle_deg)
    R = np.array([[np.cos(theta), -np.sin(theta)],
                  [np.sin(theta),  np.cos(theta)]])
    pts_rot = np.dot(pts, R.T)
    pts_trans = pts_rot + np.array(center, dtype=np.float32)
    pts_trans = np.int32(pts_trans).reshape((-1, 1, 2))
    cv2.fillPoly(frame, [pts_trans], color)

    return frame

Each crop image is processed one by one in sorted order so that the UAV's flight path appears in the correct sequence. Every crop is converted to grayscale to simplify the feature detection process and reduce computation, since color information isn’t needed for matching.

To identify where each crop fits into the global map, keypoints and descriptors are extracted using the SIFT algorithm.

FLANN is used to match crop features with those of the global map, since it performs well with high-dimensional data and large descriptor sets. Lowe’s ratio test is applied to remove unreliable matches and keep only the best ones.

Once good matches are collected, a homography is calculated using RANSAC to project the crop onto the global map. This step helps determine both the position and orientation of the drone at the time the image was captured. From the homography, the rotation angle is computed using the top edge of the crop image.

The match quality is evaluated by calculating the percentage of inliers from the RANSAC result, which helps assess how confident the model is about the match.

For visualization, a zoomed-in region around the projected crop location is extracted from the global map. This region is annotated with the crop's contour so it’s easy to see how well it aligns. The center of this region is used to track the UAV’s position over time.

In [7]:
# Process each crop image: matching, homography, rotation, similarity, visualization, and video frame creation
for idx, (crop_path, crop) in enumerate(zip(crop_files, drone_crops)):

    crop_number = idx  # Use the sorted index as the crop number
    crop_gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    kp_crop, des_crop = sift.detectAndCompute(crop_gray, None)  # Detect crop features

    # Find matches between crop and global map using FLANN with k=2
    matches = flann.knnMatch(des_crop, des_global, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    print(f"Crop {crop_number}: {len(good_matches)} good matches found.")

    if len(good_matches) > MIN_MATCH_COUNT:
        # Prepare matching points for homography calculation
        src_pts = np.float32([kp_crop[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_global[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        # Compute the homography matrix using RANSAC to filter outliers
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        # Define the four corners of the crop image and transform them using homography
        h_crop, w_crop = crop_gray.shape
        pts = np.float32([[0, 0],
                          [0, h_crop - 1],
                          [w_crop - 1, h_crop - 1],
                          [w_crop - 1, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, H)

        # Compute the rotation angle based on the transformation of the top edge
        pt_origin = np.array([[0, 0]], dtype=np.float32).reshape(1, 1, 2)
        pt_x = np.array([[w_crop - 1, 0]], dtype=np.float32).reshape(1, 1, 2)
        trans_origin = cv2.perspectiveTransform(pt_origin, H)[0][0]
        trans_pt_x = cv2.perspectiveTransform(pt_x, H)[0][0]
        dx = trans_pt_x[0] - trans_origin[0]
        dy = trans_pt_x[1] - trans_origin[1]
        angle_rad = np.arctan2(dy, dx)
        angle_deg = np.degrees(angle_rad)

        # Calculate similarity percentage from inlier ratio
        if mask is not None and len(mask) > 0:
            similarity = (np.sum(mask) / len(mask)) * 100
        else:
            similarity = 0.0

        # Create a zoomed region for side-by-side visualization by finding a bounding rectangle
        x, y, w, h = cv2.boundingRect(np.int32(dst))
        x_zoom = max(x - margin, 0)
        y_zoom = max(y - margin, 0)
        x_end = min(x + w + margin, global_map.shape[1])
        y_end = min(y + h + margin, global_map.shape[0])
        zoomed_region = global_map[y_zoom:y_end, x_zoom:x_end].copy()
        dst_adjusted = dst - np.array([[x_zoom, y_zoom]], dtype=np.float32)
        zoomed_region_with_contour = cv2.polylines(zoomed_region.copy(), [np.int32(dst_adjusted)], True, (0, 255, 0), 3)

        # Compute the center (drone position) and update our trajectory list
        center = np.mean(dst, axis=0)
        center_tuple = (int(center[0][0]), int(center[0][1]))
        drone_positions.append(center_tuple)

        # Create side-by-side visualization for review (crop + zoomed global map)
        fig, axs = plt.subplots(1, 2, figsize=(15, 7))
        axs[0].imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        axs[0].set_title(f"Crop {crop_number}")
        axs[0].axis("off")
        axs[1].imshow(cv2.cvtColor(zoomed_region_with_contour, cv2.COLOR_BGR2RGB))
        axs[1].set_title("Zoomed Global Map")
        axs[1].axis("off")
        axs[1].text(10, 20, f"Rotation: {angle_deg:.1f}°", color='red', fontsize=14, backgroundcolor='white')
        axs[1].text(10, 35, f"Similarity: {similarity:.1f}%", color='red', fontsize=14, backgroundcolor='white')
        plt.tight_layout()
        output_file = os.path.join(output_folder, f'combined_zoom_output_{crop_number}.png')
        plt.savefig(output_file, bbox_inches='tight')
        plt.show()

        # Create the global map video frame
        frame = global_map.copy()
        if len(drone_positions) > 1:
            for i in range(1, len(drone_positions)):
                cv2.line(frame, drone_positions[i-1], drone_positions[i], (0, 0, 255), 2)
            cv2.arrowedLine(frame, drone_positions[-2], drone_positions[-1], (255, 255, 0), 2, tipLength=0.3)
        frame = draw_drone_triangle(frame, center_tuple, angle_deg, size=20, color=(0, 255, 255))
        cv2.putText(frame, f"Rotation: {angle_deg:.1f} deg", (center_tuple[0] + 15, center_tuple[1] + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, f"Similarity: {similarity:.1f}%", (center_tuple[0] + 15, center_tuple[1] + 55),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, f"Frame {crop_number}", (center_tuple[0] + 15, center_tuple[1] - 15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        video_writer.write(frame)  # Write the global frame to the video

        # Create the zoomed video frame by resizing the zoomed region to a fixed size
        zoom_frame = cv2.resize(zoomed_region_with_contour, zoom_frame_size)
        zoom_video_writer.write(zoom_frame)  # Write the zoomed frame to the zoomed video

    else:
        print(f"Not enough matches found for crop {crop_number} ({len(good_matches)} good matches).")

video_writer.release()
zoom_video_writer.release()
print(f"Global video saved to {video_output_file}")
print(f"Zoomed video saved to {zoom_video_output_file}")

Output hidden; open in https://colab.research.google.com to view.

## 5. Visualized Route Map

Create an extra image that shows the global map overlaid with the full drone route. Draw a polyline connecting the route points, marks each point with a small circle and order number, and explicitly label the start and finish positions.

In [11]:
# Copy the original global map to avoid overwriting it.
route_map = global_map.copy()

if drone_positions:
    # Draw the complete route as a polyline connecting all drone positions
    pts = np.array(drone_positions, dtype=np.int32)
    cv2.polylines(route_map, [pts], isClosed=False, color=(0, 0, 255), thickness=2)

    # Loop through each drone position to mark and annotate the order
    for idx, pos in enumerate(drone_positions):
        # Draw a small circle at the route point
        cv2.circle(route_map, pos, radius=5, color=(255, 0, 0), thickness=-1)
        # Annotate with the order number near the circle
        cv2.putText(route_map, str(idx), (pos[0] + 10, pos[1] + 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)

    # Mark the start point with a label 'Start'
    start_point = drone_positions[0]
    cv2.putText(route_map, "Start", (start_point[0] - 20, start_point[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), thickness=3, lineType=cv2.LINE_AA)

    # Mark the finish point with a label 'Finish'
    finish_point = drone_positions[-1]
    cv2.putText(route_map, "Finish", (finish_point[0] - 20, finish_point[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), thickness=3, lineType=cv2.LINE_AA)

    # Save the visualized route image to the output folder
    route_output_file = os.path.join(output_folder, 'route_overview.png')
    cv2.imwrite(route_output_file, route_map)
    print(f"Route overview image saved to {route_output_file}")
else:
    print("No drone positions were detected. Please ensure that the crop processing step produced valid positions.")

Route overview image saved to /content/drive/My Drive/Colab Notebooks/drone_navigation_project/output/route_overview.png


## What could be improved in future

- the trajectory could be smoothed using interpolation or filtering to remove jitter and better reflect real drone motion;
- matching could be improved by combining SIFT with other features like color histograms or semantic segmentation;
- instead of homography from a single frame, temporal information across frames could be used to improve robustness;
- adding support for handling frames with too few matches by predicting position from past trajectory would help fill gaps;
- the zoomed-in region size could adapt dynamically based on match quality or drone altitude;
- exporting data in formats like GeoJSON or CSV could help with further analysis or GIS integration.